## ANN Churn Prediction

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("../data/churn_modellings.csv")

print("Nan Values:")
print(df.isna().sum())

print("\n Duplicated values:")
print(df.duplicated().sum())

print("DataFrame Information:")
print(df.info())

Nan Values:
RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

 Duplicated values:
0
DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts   

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Dropping irrelevant features
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Splitting the data into features and target
X = df.drop(columns=["Exited"])
y = df["Exited"]

# Splitting data into train and test splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [6]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [7]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [8]:
# Getting numerical and categorical columns
categorical_columns = X_train.select_dtypes(
    include=["object", "category"]
).columns.to_list()

numerical_columns = X_train.select_dtypes(
    include=["int64", "float64"]
).columns.to_list()

for col in categorical_columns:
    values = df[col].value_counts()
    print(f"{col} values are:\n{values}\n")
print(numerical_columns)

Geography values are:
Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

Gender values are:
Gender
Male      5457
Female    4543
Name: count, dtype: int64

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [9]:
# Scaling and enconding data
ohe_feature = ["Geography"]
ordinal_feature = ["Gender"]

scaler = StandardScaler()
ohe_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
ordinal_encoder = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", scaler, numerical_columns),
        ("geography", ohe_encoder, ohe_feature),
        ("gender", ordinal_encoder, ordinal_feature),
    ],
    remainder="passthrough",
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
    ]
)

In [10]:
X_train_processed = pipeline.fit_transform(X_train)
X_test_processed = pipeline.transform(X_test)

In [11]:
X_train_processed[:5]

array([[ 0.35649971, -0.6557859 ,  0.34567966, -1.21847056,  0.80843615,
         0.64920267,  0.97481699,  1.36766974,  1.        ,  0.        ,
         0.        ,  1.        ],
       [-0.20389777,  0.29493847, -0.3483691 ,  0.69683765,  0.80843615,
         0.64920267,  0.97481699,  1.6612541 ,  0.        ,  1.        ,
         0.        ,  1.        ],
       [-0.96147213, -1.41636539, -0.69539349,  0.61862909, -0.91668767,
         0.64920267, -1.02583358, -0.25280688,  0.        ,  0.        ,
         1.        ,  1.        ],
       [-0.94071667, -1.13114808,  1.38675281,  0.95321202, -0.91668767,
         0.64920267, -1.02583358,  0.91539272,  1.        ,  0.        ,
         0.        ,  0.        ],
       [-1.39733684,  1.62595257,  1.38675281,  1.05744869, -0.91668767,
        -1.54035103, -1.02583358, -1.05960019,  1.        ,  0.        ,
         0.        ,  1.        ]])

In [12]:
X_test_processed[:2]

array([[-0.57749609, -0.6557859 , -0.69539349,  0.32993735,  0.80843615,
        -1.54035103, -1.02583358, -1.01960511,  0.        ,  1.        ,
         0.        ,  1.        ],
       [-0.29729735,  0.3900109 , -1.38944225, -1.21847056,  0.80843615,
         0.64920267,  0.97481699,  0.79888291,  1.        ,  0.        ,
         0.        ,  1.        ]])

In [13]:
X_train_processed.shape, X_test_processed.shape

((8000, 12), (2000, 12))

In [14]:
# Saving encoders and scaler
with open(file="ohe_encoder.pkl", mode="wb") as file:
    pickle.dump(ohe_encoder, file)

with open(file="ordinal_encoder.pkl", mode="wb") as file:
    pickle.dump(ordinal_encoder, file)

with open(file="scaler.pkl", mode="wb") as file:
    pickle.dump(scaler, file)

#### ANN Implementation for Churn Prediction

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [16]:
print(f"Versión de TensorFlow: {tf.__version__}")

# El comando es el mismo, pero ahora buscará un dispositivo Metal.
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print(f"✅ ¡Éxito! TensorFlow ha detectado la GPU de Metal: {gpus[0].name}")
else:
    print(
        "⚠️ No se encontró ninguna GPU. Asegúrate de haber instalado 'tensorflow-metal'."
    )

Versión de TensorFlow: 2.19.0
⚠️ No se encontró ninguna GPU. Asegúrate de haber instalado 'tensorflow-metal'.


In [17]:
# Input shape
(X_train_processed.shape[1],)

(12,)

In [18]:
# ANN creation
input_dimensions = X_train_processed.shape[1]

model = Sequential(
    [
        Input(shape=(input_dimensions,), name="input_layer"),
        Dense(64, activation="relu", name="hidden_layer_1"),
        Dense(32, activation="relu", name="hidden_layer_2"),
        Dense(1, activation="sigmoid", name="output_layer"),
    ],
    name="churn_prediction_model",
)

In [19]:
print("Model defined and compiled successfully:")
model.summary()

Model defined and compiled successfully:


Model: "churn_prediction_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Adding specific learning rate to optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

# Compile the model
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [21]:
# Setup TensorBoard

log_parent_directory = "logs/fit"

log_dir = os.path.join(log_parent_directory, datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"))
tensorflow_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
)

In [22]:
# Setup EarlyStopping
early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
)

In [23]:
# Training model
history = model.fit(
    X_train_processed,
    y_train,
    validation_data=(X_test_processed, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback],
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8069 - loss: 0.4271 - val_accuracy: 0.8605 - val_loss: 0.3564
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.8512 - loss: 0.3518 - val_accuracy: 0.8615 - val_loss: 0.3439
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - accuracy: 0.8531 - loss: 0.3436 - val_accuracy: 0.8560 - val_loss: 0.3388
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.8599 - loss: 0.3378 - val_accuracy: 0.8600 - val_loss: 0.3425
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.8638 - loss: 0.3302 - val_accuracy: 0.8530 - val_loss: 0.3438
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - accuracy: 0.8620 - loss: 0.3308 - val_accuracy: 0.8615 - val_loss: 0.3344
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.8660 - loss: 0.3297 - val_accuracy: 0.8625 - val_loss: 0.3365
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.8651 - loss: 0.3

In [ ]:
model.save("model.keras")

In [25]:
# Load TensorBoard extnesion
%load_ext tensorboard

In [28]:
%tensorboard --logdir {log_parent_directory}

Reusing TensorBoard on port 6006 (pid 85424), started 0:06:23 ago. (Use '!kill 85424' to kill it.)